## Description:
这个笔记本把前面两个流程合起来封装成一个函数，给定一张图片，最终输出识别的文字信息

In [1]:
import numpy as np
import cv2
import os

from utils import resize, cv_show, four_point_transform, get_screenCnt, get_rotated_img, ocr_check

In [2]:
def img_ocr(image):
    # resize操作之前，需要保存resize的比例，以及原始图像
    # 下面要按照500的比例对图像进行resize， 那么原始图像的每个像素点的位置都会改变，记住ratio是为了最终能还原到原始的位置上去
    ratio = image.shape[0] / 500.0
    orig = image.copy()
    
    image = resize(orig, height=500)
    
    # 预处理 转成灰度图 -> 高斯滤波 -> Canny边缘检测
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    #gray = cv2.GaussianBlur(gray, (5, 5), 0)   边缘检测算法其实就是用的高斯滤波，所以这里这个不用发现更加清晰些
    edged = cv2.Canny(gray, 75, 200)
    
    # 轮廓检测
    cnts, hierarchy = cv2.findContours(edged.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    cnts = sorted(cnts, key = cv2.contourArea, reverse=True)[:5]
    screenCnt = get_screenCnt(cnts)
    
    # 透视变换
    warped = four_point_transform(orig, screenCnt.reshape(4, 2)*ratio)
    
    # 二值处理
    warped = cv2.cvtColor(warped, cv2.COLOR_BGR2GRAY)
    ref = cv2.threshold(warped, 150, 255, cv2.THRESH_BINARY)[1]
    
    rotated_img = get_rotated_img(ref)
    resize_img = resize(rotated_img, height=600)
    # 保存到文件
    cv2.imwrite('images/scan.jpg', resize_img)
    # cv_show('res', resize_img)
    
    # ocr识别
    image = cv2.imread('images/scan.jpg')
    text = ocr_check(image)
    
    return text

In [3]:
# 读取图片
image = cv2.imread("images/receipt.jpg")  
#cv_show('img', image)     # 原始图片是2448*3264, 太大了，下面需要进行resize操作

In [4]:
text = img_ocr(image)

In [7]:
print(text)

WHOLE FOODS MARKET - WESTPORT, CT 06880
399 POST RD WEST - (203) 227-6858

.99
.99

365 - BACON LS NP 4

365 _ BACON LS NP 4
365 _ BACON LS NP 4.99
365 _ BACON LS NP 4.99
BROTH CHIC NP 2.19
FLOUR ALMOND NP 11.99
CHKN BRST BNLSS SK NP 18.80
HEAVY CREAM NP _ 3.39
BALSMC REDUCT NP _ 6.49
BEEF GRND 85/15 NP - 5.04

  

wow wow wow wow ow ok


